In [4]:
import sys
print(sys.executable)
!{sys.executable} -m pip install torch torch-vision nltk bert_score accelerate

!{sys.executable} -m pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

/opt/anaconda3/bin/python


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: bundlerextension console dejavu events execute kernel
kernelspec migrate nbclassic nbconvert nbextension notebook qtconsole run
server serverextension troubleshoot trust

Jupyter command `jupyter-labextension` not found.


In [5]:
!jupyter nbextension install --py widgetsnbextension --sys-prefix
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

Installing /opt/homebrew/lib/python3.10/site-packages/widgetsnbextension/static -> jupyter-js-widgets
Making directory: /opt/homebrew/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/share/jupyter/nbextensions/jupyter-js-widgets/
Copying: /opt/homebrew/lib/python3.10/site-packages/widgetsnbextension/static/extension.js.LICENSE.txt -> /opt/homebrew/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/share/jupyter/nbextensions/jupyter-js-widgets/extension.js.LICENSE.txt
Copying: /opt/homebrew/lib/python3.10/site-packages/widgetsnbextension/static/extension.js.map -> /opt/homebrew/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/share/jupyter/nbextensions/jupyter-js-widgets/extension.js.map
Copying: /opt/homebrew/lib/python3.10/site-packages/widgetsnbextension/static/extension.js -> /opt/homebrew/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/share/jupyter/nbextensions/jupyter-js-widgets/extension.js
- Validating: OK

    To initialize this nbextension i

# Text Generation: Theory and Practice

### 1. Decoding Strategies
When language models generate text, they predict probability distributions over vocabulary at each step. Decoding strategies determine how we select the next token from these distributions.

### 1.1 Greedy Decoding
Selects the highest probability token at each step.
- Pros: Fast, deterministic
- Cons: Can miss better overall sequences, repetitive output

In [7]:
import torch
import torch.nn.functional as F

def greedy_decode(logits):
    """
    logits: tensor of shape (vocab_size,) with unnormalized scores
    """
    probs = F.softmax(logits, dim=-1)
    next_token = torch.argmax(probs, dim=-1)
    return next_token

# Example
vocab_size = 1000
logits = torch.randn(vocab_size)
token = greedy_decode(logits)
print(f"Selected token ID: {token.item()}")

Selected token ID: 414


In [8]:
### 1.2 Sampling Methods
### Random Sampling
### Sample from the full probability distribution.

def random_sampling(logits, temperature=1.0):
    """
    temperature: controls randomness (higher = more random)
    """
    probs = F.softmax(logits / temperature, dim=-1)
    next_token = torch.multinomial(probs, num_samples=1)
    return next_token

# Example with different temperatures
logits = torch.randn(1000)
print("Low temp (0.5):", random_sampling(logits, 0.5).item())
print("High temp (2.0):", random_sampling(logits, 2.0).item())

Low temp (0.5): 6
High temp (2.0): 907


In [11]:
# Top-k Sampling
# Sample from only the k most likely tokens.

def top_k_sampling(logits, k=50, temperature=1.0):
    """
    k: number of top tokens to consider
    """
    # Get top k logits and indices
    top_k_logits, top_k_indices = torch.topk(logits, k)
    
    # Apply temperature and softmax
    probs = F.softmax(top_k_logits / temperature, dim=-1)
    
    # Sample from top k
    next_token_idx = torch.multinomial(probs, num_samples=1)
    next_token = top_k_indices[next_token_idx]
    
    return next_token

# Example
logits = torch.randn(1000)
token = top_k_sampling(logits, k=50)
print(f"Top-k sampled token: {token.item()}")

Top-k sampled token: 306


In [12]:
# Top-p (Nucleus) Sampling
# Sample from the smallest set of tokens whose cumulative probability exceeds p.

def top_p_sampling(logits, p=0.9, temperature=1.0):
    """
    p: cumulative probability threshold (0 < p <= 1)
    """
    # Apply temperature
    probs = F.softmax(logits / temperature, dim=-1)
    
    # Sort probabilities in descending order
    sorted_probs, sorted_indices = torch.sort(probs, descending=True)
    
    # Calculate cumulative probabilities
    cumsum_probs = torch.cumsum(sorted_probs, dim=-1)
    
    # Find cutoff index where cumsum exceeds p
    mask = cumsum_probs <= p
    mask[0] = True  # Always include the top token
    
    # Filter probabilities
    filtered_probs = sorted_probs * mask
    filtered_probs = filtered_probs / filtered_probs.sum()  # Renormalize
    
    # Sample
    next_token_idx = torch.multinomial(filtered_probs, num_samples=1)
    next_token = sorted_indices[next_token_idx]
    
    return next_token

# Example
logits = torch.randn(1000)
token = top_p_sampling(logits, p=0.9)
print(f"Top-p sampled token: {token.item()}")

Top-p sampled token: 581


In [13]:
# 1.3 Beam Search
# Maintains k most probable sequences (beams) at each step.

from dataclasses import dataclass
from typing import List

@dataclass
class Beam:
    tokens: List[int]
    score: float

def beam_search(model, start_token, max_length=20, beam_width=5, vocab_size=1000):
    """
    Simplified beam search implementation
    model: callable that takes token sequence and returns logits
    """
    # Initialize with start token
    beams = [Beam(tokens=[start_token], score=0.0)]
    
    for _ in range(max_length):
        candidates = []
        
        for beam in beams:
            # Get logits for next token (mock with random for demo)
            logits = torch.randn(vocab_size)
            log_probs = F.log_softmax(logits, dim=-1)
            
            # Get top k tokens
            top_k_log_probs, top_k_tokens = torch.topk(log_probs, beam_width)
            
            # Create new beams
            for log_prob, token in zip(top_k_log_probs, top_k_tokens):
                new_beam = Beam(
                    tokens=beam.tokens + [token.item()],
                    score=beam.score + log_prob.item()
                )
                candidates.append(new_beam)
        
        # Keep top beam_width candidates
        beams = sorted(candidates, key=lambda x: x.score, reverse=True)[:beam_width]
    
    return beams[0].tokens  # Return best sequence

# Example usage
result = beam_search(None, start_token=1, max_length=10, beam_width=3)
print(f"Beam search result: {result}")

Beam search result: [1, 746, 196, 305, 648, 151, 526, 996, 46, 226, 802]


$$
\begin{array}{|l|l|l|l|l|}
\hline
\textbf{Strategy} & \textbf{Quality} & \textbf{Diversity} & \textbf{Speed} & \textbf{Use Case} \\
\hline
\text{Greedy} & \text{Moderate} & \text{Low} & \text{Fast} & \text{Factual tasks} \\
\hline
\text{Top-k} & \text{Good} & \text{High} & \text{Moderate} & \text{Creative writing} \\
\hline
\text{Top-p} & \text{Good} & \text{High} & \text{Moderate} & \text{General purpose} \\
\hline
\text{Beam} & \text{High} & \text{Low} & \text{Slow} & \text{Translation, summarization} \\
\hline
\end{array}
$$

# Use Case-1: Summarization

In [14]:
from transformers import pipeline

# Using Hugging Face transformers
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

text = """
Artificial intelligence has made remarkable progress in recent years, 
with breakthroughs in natural language processing, computer vision, and 
reinforcement learning. Large language models like GPT have demonstrated 
impressive abilities in understanding and generating human-like text. 
These advances have led to practical applications in healthcare, finance, 
education, and many other domains. However, challenges remain in areas 
such as explainability, bias mitigation, and ensuring AI systems are 
safe and aligned with human values.
"""

# Abstractive summarization with beam search
summary = summarizer(
    text, 
    max_length=50, 
    min_length=20, 
    do_sample=False,  # Use beam search
    num_beams=4
)

print("Original length:", len(text.split()))
print("Summary:", summary[0]['summary_text'])
print("Summary length:", len(summary[0]['summary_text'].split()))

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use mps:0


Original length: 71
Summary: Artificial intelligence has made remarkable progress in recent years. Large language models like GPT have demonstrated impressive abilities in understanding and generating human-like text. However, challenges remain in areas such as explainability, bias mitigation, and ensuring AI
Summary length: 37


### Key parameters for summarization:

- Use beam search for more coherent summaries
- Balance max_length and min_length
- Higher num_beams (4-8) for better quality

# Use Case-2: Text Generation

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load model
model_name = "Qwen/Qwen2.5-1.5B-Instruct"  # Smaller model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

prompt = "The future of artificial intelligence will"

# Encode input
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Different decoding strategies
with torch.no_grad():
    outputs_greedy = model.generate(
        input_ids,
        max_length=50,
        do_sample=False  # Greedy
    )
with torch.no_grad():
    outputs_sampling = model.generate(
        input_ids,
        max_length=50,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8
    )
with torch.no_grad():
    outputs_beam = model.generate(
        input_ids,
        max_length=50,
        num_beams=5,
        early_stopping=True
    )

print("Greedy:", tokenizer.decode(outputs_greedy[0], skip_special_tokens=True))
print("\nSampling:", tokenizer.decode(outputs_sampling[0], skip_special_tokens=True))
print("\nBeam:", tokenizer.decode(outputs_beam[0], skip_special_tokens=True))

Greedy: The future of artificial intelligence will be shaped by the decisions we make today. As AI becomes more prevalent in our daily lives, it is important to consider how we can ensure that it is used ethically and responsibly.
One way to do this is through

Sampling: The future of artificial intelligence will be defined by the way people choose to use it. Some say AI is a powerful tool that can help humanity achieve great things, while others worry about its potential risks and unintended consequences.
AI has already made significant strides in

Beam: The future of artificial intelligence will be shaped by the ethical and social implications of the technology. As AI becomes more prevalent in our daily lives, it is important to consider the potential consequences of its development and use. This includes the impact on employment, privacy


### Best practices:

- Creative tasks: Use sampling (top-p=0.9, temperature=0.7-1.0)
- Factual tasks: Use greedy or beam search
- Balanced: Top-p with moderate temperature (0.7-0.8)

## 3. Evaluation Metrics

### 3.1 BLEU (Bilingual Evaluation Understudy)

Measures n-gram overlap between generated and reference texts. Originally designed for machine translation.

**Formula:**

$$\text{BLEU} = BP \times \exp\left(\sum_{n=1}^{N} w_n \times \log p_n\right)$$

**Where:**
- $p_n$ = n-gram precision
- $w_n$ = weights (usually 1/4 for 1-4 grams)
- $BP$ = brevity penalty

In [7]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction

# Single sentence example
reference = [['the', 'cat', 'is', 'on', 'the', 'mat']]
candidate = ['the', 'cat', 'sat', 'on', 'the', 'mat']

# Calculate BLEU score
bleu_score = sentence_bleu(reference, candidate)
print(f"BLEU score: {bleu_score:.4f}")

# With different n-gram weights
bleu_1 = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
bleu_2 = sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0))
bleu_4 = sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25))

print(f"BLEU-1: {bleu_1:.4f}")
print(f"BLEU-2: {bleu_2:.4f}")
print(f"BLEU-4: {bleu_4:.4f}")

# Corpus-level BLEU
references = [
    [['the', 'cat', 'is', 'on', 'the', 'mat']],
    [['there', 'is', 'a', 'dog', 'in', 'the', 'park']]
]
candidates = [
    ['the', 'cat', 'sat', 'on', 'the', 'mat'],
    ['a', 'dog', 'is', 'in', 'the', 'park']
]

corpus_bleu_score = corpus_bleu(references, candidates)
print(f"Corpus BLEU: {corpus_bleu_score:.4f}")

# Using smoothing for short sentences
smooth = SmoothingFunction()
bleu_smooth = sentence_bleu(reference, candidate, 
                            smoothing_function=smooth.method1)
print(f"Smoothed BLEU: {bleu_smooth:.4f}")

BLEU score: 0.0000
BLEU-1: 0.8333
BLEU-2: 0.7071
BLEU-4: 0.0000
Corpus BLEU: 0.0000
Smoothed BLEU: 0.2541


/opt/anaconda3/lib/python3.12/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


### BLEU Characteristics:

- Range: 0 to 1 (higher is better)
- Pros: Fast, language-agnostic, correlates with human judgment
- Cons: Doesn't capture semantics, sensitive to length, requires references

### 3.2 BERTScore
* Uses contextual embeddings from BERT to compute similarity between tokens.
Components:

- Precision: Matching of candidate tokens to reference
- Recall: Coverage of reference by candidate
- F1: Harmonic mean of precision and recall

In [9]:
from bert_score import score

# Example texts
references = [
    "The cat is sitting on the mat.",
    "Dogs are playing in the park."
]

candidates = [
    "A cat sits on a mat.",
    "The dogs play at the park."
]

# Calculate BERTScore
P, R, F1 = score(candidates, references, lang="en", verbose=True)

print("\nBERTScore Results:")
for i, (p, r, f1) in enumerate(zip(P, R, F1)):
    print(f"Pair {i+1}:")
    print(f"  Precision: {p:.4f}")
    print(f"  Recall: {r:.4f}")
    print(f"  F1: {f1:.4f}")

# Using specific model
P, R, F1 = score(candidates, references, 
                 model_type="microsoft/deberta-xlarge-mnli")

# Batch processing
print(f"\nAverage F1: {F1.mean():.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.47 seconds, 1.36 sentences/sec

BERTScore Results:
Pair 1:
  Precision: 0.9688
  Recall: 0.9679
  F1: 0.9684
Pair 2:
  Precision: 0.9552
  Recall: 0.9607
  F1: 0.9579

Average F1: 0.9211


### BERTScore Characteristics:

- Range: Typically 0.5 to 1.0 (higher is better)
- Pros: Captures semantic similarity, robust to paraphrasing
- Cons: Slower than BLEU, requires GPU for efficiency, model-dependent

## Metric Selection

$$
\begin{array}{|l|l|l|}
\hline
\textbf{Task} & \textbf{Primary Metric} & \textbf{Secondary} \\
\hline
\text{Translation} & \text{BLEU} & \text{BERTScore} \\
\hline
\text{Summarization} & \text{ROUGE} & \text{BERTScore} \\
\hline
\text{Dialogue} & \text{BERTScore} & \text{Human eval} \\
\hline
\text{Paraphrase} & \text{BERTScore} & \text{BLEU} \\
\hline
\end{array}
$$